# **Multi-Modality Organ Volume Measurements in ADPKD**

---
Script author: **Xinzi He, Zhongxiu Hu, Hreedi Dev and Martin R. Prince**

---





### **Overview**
**mmUNet** is a high test-retest reliable organ volume measurement tool supporting multi-modality (**T1 Axial, T2 Axial, SSFP Axial, T2 Coronal, SSFP Coronal**) in autosomal dominant polycstic kidney disease (ADPKD) that can measure the volume of **left/right kidney, liver and spleen**. The organ volume measurement is based on deep learning segmentation methods which has been trained on 5 pulse sequence. Note this tool supports both dicom and nifti.

If you use our tool or dataset, please cite mmUNet (BibTeX):

```
Test Re-Test Reproducibility of Organ Volume Measurements in ADPKD using 3D Multi-Modality Deep Learning

Xinzi He, Zhongxiu Hu, Hreedi Dev, Dominick J. Romano, Arman Sharbatdaran, Syed I. Raza, Sophie J. Wang, Kurt Teichman, George Shih, James M. Chevalier, Daniil Shimonov, Jon D. Blumenfeld, Akshay Goel, Mert R. Sabuncu, and Martin R. Prince
```





**1. Make sure you have enable gpu runtime.**

(If GPU runtime of colab has been enabled you can see T4 on the right upper corner which is right below the share button).
(To enable GPU runtime, click Runtime->Change Runtime Type. In the poped up window, click T4 GPU/A100 GPU)

---



**2. Please Upload images to you google drive and then select input folder and output folder below.**



In [1]:
#@title Preparing
%%capture
!pip install nnunet==1.7.1
!pip install ipyfilechooser
!pip install dicom2nifti

import tempfile
import os
import shutil
import glob
import contextlib
from ipyfilechooser import FileChooser
from tqdm import notebook as tqdm
from nnunet.inference.predict import predict_from_folder,predict_cases
from google.colab import drive
import time
import dicom2nifti

os.makedirs('/content/model_checkpoint',exist_ok=True)
os.makedirs('/content/test_data',exist_ok=True)

!gdown 1xWE1tT7JUuRO2N7SrNo8uEDzJdUwLAsH
!unzip -o /content/model_checkpoint.zip -d /content/model_checkpoint
!gdown 16Xirt3XxICb6TKyxWnW5FAFOaztPrxnp
!unzip -o /content/test_data.zip -d /content/test_data
!rm -rf model_checkpoint.zip
!rm -rf test_data.zip

In [8]:
#@title Select input folder and output folder
drive.mount('/content/drive')
fc_input = FileChooser('/content/drive')
fc_input.title = '<b>Please Select folder before you move forward</b>'
display(fc_input)
fc_output = FileChooser('/content/drive/MyDrive')
fc_output.title ='<b>Please select output folder. The output file name is adpkd_segmentation.zip</b>'
display(fc_output)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileChooser(path='/content/drive', filename='', title='<b>Please Select folder before you move forward</b>', s…

FileChooser(path='/content/drive/MyDrive', filename='', title='<b>Please select output folder. The output file…

In [11]:
#@title Process selected folder
%%capture

assert fc_input.selected_path is not None
assert fc_output.selected_path is not None

if os.path.isdir(fc_input.selected_path):
    user_input_path = fc_input.selected_path
elif os.path.isfile(fc_input.selected_path):
    user_input_path = os.path.dirname(fc_input.selected_path)
else:
    print("It is not file or dir" )

if os.path.isdir(fc_output.selected_path):
    final_output_path = fc_output.selected_path
elif os.path.isfile(fc_output.selected_path):
    final_output_path = os.path.dirname(fc_output.selected_path)
else:
    print("It is not file or dir" )
os.makedirs('/content/images',exist_ok=True)
dicom2nifti.convert_directory(user_input_path, '/content/images', compression=True, reorient=True)
nii_gz_file_path = glob.glob(os.path.join(user_input_path,'*.nii*'))
for i in nii_gz_file_path:
  shutil.copy2(i, os.path.join('/content/images',os.path.basename(i)))

In [ ]:
#@title Segment images

base_path=os.getcwd()
with tempfile.TemporaryDirectory(dir=base_path):
  input_folder = os.path.join(base_path,'image_input_folder')
  os.makedirs(input_folder,exist_ok=True)
  output_folder = os.path.join(base_path,'seg_output_folder')
  os.makedirs(output_folder,exist_ok=True)
  nii_file_list = glob.glob('/content/images/*.nii.gz')
  if len(nii_file_list) ==0:
    print('There is no dicom files and nifti files, mmUNet will run on our test data')
    nii_file_list = glob.glob('/content/test_data/*.nii.gz')
  # for ind,file_path in enumerate(nii_file_list):
  #   shutil.copy(file_path,os.path.join(input_input_folder, 'ADPKD_{0:03d}_0000.nii.gz'.format(ind)))
  #   print(os.path.join(input_input_folder, 'ADPKD_{0:03d}_0000.nii.gz'.format(ind)))
  output_path_list = [os.path.join(output_folder,os.path.basename(i)) for i in nii_file_list]
  input_output_pair = dict(zip(nii_file_list,output_path_list))
  for input_path, output_path in tqdm.tqdm(input_output_pair.items()):
    with contextlib.redirect_stdout(None):
      predict_cases('/content/model_checkpoint',[[input_path]], [output_path], num_threads_preprocessing=6, num_threads_nifti_save=2,
                                      do_tta=True, mixed_precision=True, overwrite_existing=True,
                                      all_in_gpu=None,
                                      step_size=0.5, checkpoint_name='model_final_checkpoint',
                                      segmentation_export_kwargs=None,
                                      disable_postprocessing=True,folds='all',save_npz=False);
  shutil.make_archive('/content/adpkd_segmentation','zip',output_folder)
  shutil.copy2('/content/adpkd_segmentation.zip',os.path.join(final_output_path,'adpkd_segmentation.zip'))


There is no dicom files and nifti files, mmUNet will run on our test data


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
#@title Generate report
%matplotlib inline

import SimpleITK as sitk
import seaborn as sns
import matplotlib.pyplot as plt

row_label  = [os.path.basename(i).replace('.nii.gz','') for i in output_path_list]
volume_list = []
for path in output_path_list:
  volume_size = []
  seg_sitk = sitk.ReadImage(path)
  shape_stats = sitk.LabelShapeStatisticsImageFilter()
  shape_stats.Execute(seg_sitk)

  for i in range(1,5):
    try:
      volume_size.append(round(shape_stats.GetPhysicalSize(i)/1000))
    except:
      volume_size.append(0)

  volume_list.append(volume_size)

fig, (ax_distplot) = plt.subplots(1, 1, figsize=(8, 6))
fig.suptitle('The volume measurement results (ml)', fontsize=14)

sns.set(font_scale=1)  # crazy big
volume_list_for_plot= volume_list
for row, data in enumerate(volume_list_for_plot):
    volume_list_for_plot[row].insert(0, row_label[row])
unique_classes_for_plot = ['left kidney', 'right kidney','spleen','liver']
unique_classes_for_plot.insert(0,'series_name')

selected_volume_table = plt.table(cellText=volume_list_for_plot, colLabels=unique_classes_for_plot, loc='upper left', cellLoc='center')
selected_volume_table.auto_set_column_width(col=list(range(len(unique_classes_for_plot))))
plt.axis('off')
plt.tight_layout()
plt.show()